### Deteccion de vehiculo en área de interes y prediccion con etiqueta

In [1]:
import cv2
import time
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np

In [16]:


# Load car classifier
car_classifier = cv2.CascadeClassifier('../datasets/cars.xml')

# Load video file
cap = cv2.VideoCapture('videos/mer1_mod.mp4')

# Set video FPS to 20

cap.set(cv2.CAP_PROP_FPS,1)

fps = cap.get(cv2.CAP_PROP_FPS)
print("fps:",fps)
#load model
modelt = load_model("../nn_model_vgg_19_v2_200.h5")
names = ['bus', 'car', 'truck']
x = 1
# Loop over the frames from the video
while cap.isOpened():
    # Read the next frame from the video
    ret, frame = cap.read()
    
    if not ret:
        break
    if x%3==0:

        # Detect cars in the frame
        # Define las coordenadas del ROI
        x1, y1 = 100, 550
        x2, y2 = 720, 750

        # Dibuja un rectángulo sobre la imagen original
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, ), 2)
        #Region of interest
        roi = frame[y1:y2,x1:x2]


        # Convert the ROI to grayscale
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        
        # Detect cars in the frame
    
        cars = car_classifier.detectMultiScale(gray_roi, scaleFactor=1.05, minNeighbors=5, minSize=(50, 50))

        
        # Draw bounding boxes and labels around the vehicles 
        

        # Draw bounding boxes around the cars
        for (x,y,w,h) in cars:

        
            #definir la posicion de la etiqueta
            posicion_etiqueta = (x+x1 , y+y1 - 10)
            #Dibujar el rectangulo de l deteccion
            cv2.rectangle(frame, (x+x1, y+y1), (x+x1+w, y+y1+h), (0, 255, 255), 2)

            vehicle_image_to_classifier = roi[y+y1:y+y1+h, x+x1:x+x1+w]#Imagen a clasificar

            vehicle_image_to_classifier = frame[y:y+h, x:x+w]#Imagen a clasificar

            #clasificar la seleccion:
            imaget=cv2.resize(vehicle_image_to_classifier, (224, 224), interpolation = cv2.INTER_AREA)
            
            xt = np.asarray(imaget)
            xt = preprocess_input(xt)
            xt = np.expand_dims(xt,axis=0)
            preds = modelt.predict(xt)

            #Colocar etiqueta
            cv2.putText(frame, names[np.argmax(preds)], posicion_etiqueta, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    x += 1

    # Display the resulting frame
    cv2.imshow('frame', frame)
            
    # Wait for user input to move to the next frame
    if cv2.waitKey(1) == ord('q'):
                break
        


            

            

    x +=1
# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

fps: 30.0
1/1 [==============================] - 0s 365ms/step
